In [9]:
import os, subprocess, json
from datetime import datetime
from pyarrow import fs
import pyarrow as pa
import pyspark
from pyspark.sql import SparkSession

In [16]:
def connect_hdfs(hdfs_info):
    user = hdfs_info["user"]
    host = hdfs_info["host"]
    port = hdfs_info["port"]
    
    try:
        classpath = subprocess.Popen([hdfs_info["hdfs_path"], "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
        os.environ["CLASSPATH"] = classpath.decode("utf-8")
        hdfs = fs.HadoopFileSystem(host=hdfs_info["host"], port=hdfs_info["port"], user=hdfs_info["user"])
        
        return hdfs
    except Exception as e:
        #print(f"Failed to connect hdfs {user}@{host}:{port}")
        #log(f"Failed to connect hdfs {user}@{host}:{port}", 1)
        return None


def compare_file_date(filename):
    # 파일명에서 날짜 추출 (예: kbs_2024-07-04_0012.csv)
    try:
        file_date_str = filename.split('_')[1]  # "2024-07-04"
        file_date = datetime.strptime(file_date_str, "%Y-%m-%d").date()  # datetime.date(2024, 7, 4)
        
        # 현재 날짜 가져오기
        current_date = datetime.now().date()  # 현재 날짜 (예: datetime.date(2024, 7, 4))
        
        # 날짜 비교
        if file_date == current_date:
            return True
        else:
            return False
    except Exception as e:
        return False


def get_file_list_from_hdfs(hdfs, hdfs_path):
    # HDFS 경로에서 파일 목록 가져오기
    try:
        file_infos = hdfs.get_file_info(pa.fs.FileSelector(hdfs_path, recursive=False))
        # if not file_info_list.is_directory:
        #     raise Exception(f"{hdfs_path} is not a directory")
        
        file_list = [file_info.path for file_info in file_infos]
        
        return file_list
    except Exception as e:
        print(f"Error getting file list from HDFS: {e}")
    
def filter_files_by_date(file_list):
    filtered_files = [file for file in file_list if compare_file_date(file)]
    return filtered_files

In [11]:
hdfs_info_path = "./API_KEYS/HDFS_INFO.json"
target_hdfs_dir_path = '/P3T5'
with open(hdfs_info_path, 'r') as header_f:
    hdfs_info = json.load(header_f)

conf = pyspark.SparkConf() \
            .setAppName("hdfs2db") \
            .setMaster("spark://master:7077") \
            .set("spark.blockManager.port", "10025") \
            .set("spark.driver.blockManager.port", "10026") \
            .set("spark.driver.port", "10027") \
            .set("spark.cores.max", "2")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
hdfs_connection = connect_hdfs(hdfs_info)

In [12]:
spark

In [8]:
#spark
spark.stop()

In [17]:
total_file_list = get_file_list_from_hdfs(hdfs_connection, target_hdfs_dir_path)
target_file_list = filter_files_by_date(total_file_list)

Error getting file list from HDFS: 'pyarrow._fs.FileInfo' object has no attribute 'is_directory'


TypeError: 'NoneType' object is not iterable

In [44]:
if target_file_list:
    df_list = [spark.read.csv(f"hdfs://{target_hdfs_dir_path}/{file}", header=True, inferSchema=True) for file in target_file_list]
    combined_df = df_list[0]
    for df in df_list[1:]:
        combined_df = combined_df.union(df)
    
    # 결과 DataFrame 보여주기
    combined_df.show()
else:
    print("No files found for the current date.")

In [45]:
spark_df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Contents: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Organizer: string (nullable = true)



In [ ]:
spark_df.limit(5).show()